In [10]:
from datetime import datetime
def str2time(s):
    ans = None
    if s.startswith('今天'):
        ans = datetime.today()
        t = datetime.strptime(s.split(' ')[1], '%H:%M')
        ans = ans.replace(hour = t.hour, minute = t.minute)
        return ans
    try:
        ans = datetime.strptime(s, '%m月%d日 %H:%M')
        ans = ans.replace(year = datetime.today().year)
        return ans
    except:
        try:
            ans = datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
            return ans
        except:
            print(s)
            return None

In [11]:
def get_features(user):
    vector = {
        'reputation'            : 0,
        'mention'               : 0,
        'hashtag'               : 0,
        'url'                   : 0,
        'text_similarity'       : 0,
        'time_interval_mean'    : 0,
        'time_interval_var'     : 0,
        'post_num'              : 0,
        'active_day_ratio'      : 0,
        'follow_ratio'          : 0,
        'figure_RRT'            : 0,
        'average_comm'          : 0,
        'followee_num'          : 0,
        'figure_at_every'       : 0,
        'day_interval_variance' : 0,
        'late_night_times'      : 0,
        'figure_at_sigle'       : 0,
        'figure_at'             : 0
    }
    
    # 声望：关注他的人与他关注的人之比
    try:
        vector['reputation'] = float(user['followers_num']) / float(user['following_num'])
    except:
        return dict()
    
    import jieba
    # '@'次数和'#'次数：
    vector['mention'] = 0
    vector['hashtag'] = 0
    vector['url']     = 0
    posts   = []
    time_intervals    = []
    prev_time         = None
    for post in user['weibo']:
        if '@' in post['content']:
            vector['mention'] += 1
        if '#' in post['content']:
            vector['hashtag'] += 1
        if 'http://' in post['content']:
            vector['url'] += 1
        posts.append(list(jieba.cut(post['content'])))
        
        curr_time = str2time(' '.join(post['time'].split()[:2]))
        if curr_time == None:
            continue
        if prev_time != None:
            time_intervals.append((prev_time - curr_time).seconds)
        prev_time = curr_time
    
    import numpy as np
    vector['time_interval_mean'] = np.mean(time_intervals)
    vector['time_interval_var'] = np.var(time_intervals)
    
    vector['post_num'] = int(user['weibos_num'])
    
    return vector

    from gensim.models import word2vec
    # posts之间的文本相似度
    words = word2vec.Text8Corpus('text.txt') #暂时还没有语料
    model = word2vec.Word2Vec(words, size=200)
    sum = .0
    num = 0
    for i in range(len(posts)):
        for j in range(i + 1, len(posts)):
            sum += model.similarity(posts[i], posts[j])
            num += 1
    vector['test_similarity'] = sum / num
    
    return vector

In [12]:
import os
import re
import json

def get_label(text, id):
    p = re.compile('%s.+?(yes|no)' % id)
    return re.findall(p, text)[0]
    

path = './spider/weibo/'
files = os.listdir(path)
text = open('./data/spammer_order.csv', 'r').read()

vectors = []
for file in files:
    user = json.load(open(os.path.join(path, file), 'r'))
    if int(user['weibos_num']) <= 0:
        continue
    id = file.split('.')[0]
    label = get_label(text, id)
    vector = [id] + list(get_features(user).values()) + [label]
    vectors.append(vector)

28分钟前 来自iPhone
31分钟前 来自iPhone
40分钟前 来自三星Galaxy_SII
53分钟前 来自三星Galaxy_SII
27分钟前 来自iPhone
29分钟前 来自微博
9分钟前 来自iPhone
10分钟前 来自iPhone
56分钟前 来自华为P9手机摄影再突破


/home/lzc/Common/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:94: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/lzc/Common/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:116: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


41分钟前 来自iPhone
33分钟前 来自OPPO
35分钟前 来自OPPO
29分钟前 来自荣耀8青春版
48分钟前 来自荣耀8青春版
41分钟前 来自火山短视频
53分钟前 来自iPhone
23分钟前 来自荣耀V8
33分钟前 来自360安全浏览器
26分钟前 来自一直播
32分钟前 来自一直播
40分钟前 来自一直播


In [13]:
for v in vectors:
    print(v)

['3944654540', 0.2043956043956044, 0, 38, 0, 0, 0, 26378.770833333332, 371, 3, 7, 0, 0, 0, 0, 0, 799514412.17664945, 0, 0, 'yes']
['5055635073', 0.13675675675675675, 0, 0, 0, 0, 0, 37345.510204081635, 467, 1, 0, 0, 0, 0, 0, 0, 116925099.92336527, 0, 0, 'yes']
['2703197560', 0.3020833333333333, 0, 4, 0, 0, 0, 14511.469387755102, 401, 42, 4, 0, 0, 0, 0, 0, 603660650.16743016, 0, 0, 'yes']
['1223815707', 3.4933701657458562, 0, 18, 0, 0, 0, 33078.620000000003, 1479, 4, 12, 0, 0, 0, 0, 0, 1000798343.7556, 0, 0, 'yes']
['2028076091', 1.2808988764044944, 0, 0, 0, 0, 0, 34817.102040816324, 644, 2, 1, 0, 0, 0, 0, 0, 694379140.58142436, 0, 0, 'yes']
['3166303451', 0.4398976982097187, 0, 2, 0, 0, 0, 17949.0, 722, 25, 32, 0, 0, 0, 0, 0, 710879227.125, 0, 0, 'yes']
['2766284531', 1.0, 0, 1, 0, 0, 0, 65396.666666666664, 8, 0, 0, 0, 0, 0, 0, 0, 286048049.22222227, 0, 0, 'yes']
['5340324549', 0.3969631236442516, 0, 0, 0, 0, 0, 21691.979591836734, 631, 0, 33, 0, 0, 0, 0, 0, 226848036.26488963, 0, 0, 'y

In [16]:
import json
with open('./feature.json', 'w') as myfile:
    json.dump(vectors, myfile)